In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("Womens Clothing E-Commerce Reviews.csv")

In [ ]:
df=df.drop(columns="Unnamed: 0")

In [ ]:
df.columns=df.columns.str.lower().str.replace(" ","_")

In [ ]:
df

,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [ ]:
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure required nltk packages are downloaded
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Load the data into a DataFrame


# Preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Removing stopwords and lemmatizing
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]

    return ' '.join(tokens)


df['review_text'] = df['review_text'].fillna('')



# Apply preprocessing to the review text
df['processed_text'] = df['review_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df['processed_text']

,processed_text
0,absolutely wonderful silky sexy comfortable
1,love dress sooo pretty happened find store gla...
2,high hope dress really wanted work initially o...
3,love love love jumpsuit fun flirty fabulous ev...
4,shirt flattering due adjustable front tie perf...
...,...
23481,happy snag dress great price easy slip flatter...
23482,reminds maternity clothes soft stretchy shiny ...
23483,fit well top see never would worked glad able ...
23484,bought dress wedding summer cute unfortunately...


In [ ]:
# Preparing the data: features (X) and target (y)
X = df['processed_text']
y = df['recommended_ind']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Vectorizing the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

from sklearn.metrics import classification_report, accuracy_score

# Training a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Making predictions on the test set
y_pred = nb_classifier.predict(X_test_tfidf)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_output)



Accuracy: 0.8535547041294168
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.21      0.33       835
           1       0.85      0.99      0.92      3863

    accuracy                           0.85      4698
   macro avg       0.86      0.60      0.63      4698
weighted avg       0.86      0.85      0.81      4698



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Step 5: Train a Model (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Step 6: Make Predictions and Evaluate the Model
y_pred = rf_model.predict(X_test_tfidf)

# Accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8654746700723712
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.33      0.46       835
           1       0.87      0.98      0.92      3863

    accuracy                           0.87      4698
   macro avg       0.83      0.65      0.69      4698
weighted avg       0.86      0.87      0.84      4698



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Defining the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initializing the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Setting up RandomizedSearchCV
rf_random_search = RandomizedSearchCV(estimator=rf_classifier,
                                      param_distributions=param_grid,
                                      n_iter=50,
                                      cv=3,
                                      verbose=2,
                                      random_state=42,
                                      n_jobs=-1)

# Fitting the random search model
rf_random_search.fit(X_train_tfidf, y_train)

# Extracting the best parameters and the best model
best_params = rf_random_search.best_params_
best_rf_model = rf_random_search.best_estimator_

# Making predictions on the test set with the best model
y_pred_best_rf = best_rf_model.predict(X_test_tfidf)

# Evaluating the tuned model
best_rf_accuracy = accuracy_score(y_test, y_pred_best_rf)
best_rf_classification_report = classification_report(y_test, y_pred_best_rf)

print("Best Parameters:", best_params)
print("Best Model Accuracy:", best_rf_accuracy)
print("Classification Report:\n", best_rf_classification_report)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': False}
Best Model Accuracy: 0.8688803746275011
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.38      0.51       835
           1       0.88      0.97      0.92      3863

    accuracy                           0.87      4698
   macro avg       0.82      0.68      0.72      4698
weighted avg       0.86      0.87      0.85      4698



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Parameters
max_words = 10000  # Maximum number of words to keep in the vocabulary
max_len = 100  # Maximum length of a sequence

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_len, padding='post', truncating='post')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42, stratify=y)

# LSTM Model Definition
model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print("Test Accuracy:", test_accuracy)
